# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp (F),Humidity,Cloudiness,Wind Speed,Country,Date
0,0,chonchi,-42.6128,-73.8092,51.12,84,17,3.83,CL,1704852783
1,1,blagoveshchenka,52.8333,79.8667,4.37,82,100,16.42,RU,1704852783
2,2,penalva,-3.2942,-45.1736,73.81,90,20,2.57,BR,1704852784
3,3,invercargill,-46.4000,168.3500,76.01,55,100,10.00,NZ,1704852784
4,4,susuman,62.7833,148.1667,-31.77,98,100,1.72,RU,1704852785


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [5]:
# Configure the map plot
map_plot1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    size="Humidity",
    color="orange",
    line_color="black",
    tiles = "OSM")

# Display the map
map_plot1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [6]:
# Narrow down cities that fit criteria and drop any results with null values
criteria_df = city_data_df[(city_data_df["Max Temp (F)"] < 80) & (city_data_df["Max Temp (F)"] > 40) &
                        (city_data_df["Wind Speed"] < 6) & (city_data_df["Cloudiness"] == 0)]

# Drop any rows with null values
filtered_df = criteria_df.dropna()

# Display sample data
filtered_df

,City_ID,City,Lat,Lng,Max Temp (F),Humidity,Cloudiness,Wind Speed,Country,Date
12,12,isla vista,34.4133,-119.8610,58.35,72,0,3.44,US,1704852789
19,19,puerto penasco,31.3167,-113.5333,53.91,37,0,1.01,MX,1704852793
94,94,ginir,7.1333,40.7000,53.51,58,0,3.24,ET,1704852831
104,104,coquimbo,-29.9533,-71.3436,60.53,88,0,3.44,CL,1704852835
146,146,howrah,22.5892,88.3103,57.20,88,0,0.00,IN,1704852857
289,289,puruliya,23.3333,86.3667,63.59,50,0,2.46,IN,1704852941
295,295,adam,22.3756,57.5233,56.21,54,0,1.36,OM,1704852945
323,323,tezu,27.9167,96.1667,61.86,37,0,0.38,IN,1704852959
372,372,tura,25.5198,90.2201,58.89,52,0,2.55,IN,1704852984
397,397,chabahar,25.2919,60.6430,66.79,41,0,4.68,IR,1704853001


### Step 3: Create a new DataFrame called `hotel_df`.

In [7]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = filtered_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
12,isla vista,US,34.4133,-119.8610,72,
19,puerto penasco,MX,31.3167,-113.5333,37,
94,ginir,ET,7.1333,40.7000,58,
104,coquimbo,CL,-29.9533,-71.3436,88,
146,howrah,IN,22.5892,88.3103,88,
289,puruliya,IN,23.3333,86.3667,50,
295,adam,OM,22.3756,57.5233,54,
323,tezu,IN,27.9167,96.1667,37,
372,tura,IN,25.5198,90.2201,52,
397,chabahar,IR,25.2919,60.6430,41,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [8]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "limit":20,
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lng = row["Lng"]
    lat = row["Lat"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
isla vista - nearest hotel: The Club & Guest House
puerto penasco - nearest hotel: Hotel Paraíso Del Desierto
ginir - nearest hotel: No hotel found
coquimbo - nearest hotel: Hotel Iberia
howrah - nearest hotel: Sun India Guest House
puruliya - nearest hotel: No hotel found
adam - nearest hotel: No hotel found
tezu - nearest hotel: No hotel found
tura - nearest hotel: No hotel found
chabahar - nearest hotel: هتل سپیده
khamis mushait - nearest hotel: No hotel found
suez - nearest hotel: فندق لوسيندا
iquitos - nearest hotel: Marfil del Amazonas
al jawf - nearest hotel: No hotel found
corfu - nearest hotel: Hotel Bella Venezia
medina - nearest hotel: The Oberoi Madina
long beach - nearest hotel: Westin


,City,Country,Lat,Lng,Humidity,Hotel Name
12,isla vista,US,34.4133,-119.8610,72,The Club & Guest House
19,puerto penasco,MX,31.3167,-113.5333,37,Hotel Paraíso Del Desierto
94,ginir,ET,7.1333,40.7000,58,No hotel found
104,coquimbo,CL,-29.9533,-71.3436,88,Hotel Iberia
146,howrah,IN,22.5892,88.3103,88,Sun India Guest House
289,puruliya,IN,23.3333,86.3667,50,No hotel found
295,adam,OM,22.3756,57.5233,54,No hotel found
323,tezu,IN,27.9167,96.1667,37,No hotel found
372,tura,IN,25.5198,90.2201,52,No hotel found
397,chabahar,IR,25.2919,60.6430,41,هتل سپیده


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [9]:
%%capture --no-display

# Configure the map plot
map_plot2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    size = "Humidity",
    scale = 1,
    color= "City",
    line_color="black",
    tiles = "OSM",
    frame_width = 500,
    frame_height = 400,
    legend = True,
    hover_cols=["Country", "Hotel Name"]
)
# Display the map
map_plot2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)